# 1. Import Dependencies

In [2]:
import gym
import numpy as np
import os
import random

from gym import Env
from gym.spaces import Discrete, Box, Dict, MultiBinary, MultiDiscrete, Tuple
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy


# 2. Types of Spaces

In [3]:
Discrete(3).sample()

1

In [4]:
Box(0, 1, shape=(3,)).sample()

array([0.11960221, 0.92005783, 0.7712477 ], dtype=float32)

In [5]:
Tuple((Discrete(3), Box(0, 1, shape=(3,)))).sample()

(0, array([0.29003108, 0.3365333 , 0.9768273 ], dtype=float32))

In [10]:
Dict({'height':Discrete(2), 'speed': Box(0,100, shape=(1,)), 'color': MultiBinary(4)}).sample()

OrderedDict([('color', array([1, 0, 1, 1], dtype=int8)),
             ('height', 0),
             ('speed', array([45.566723], dtype=float32))])

In [7]:
MultiBinary(4).sample()

array([1, 0, 1, 1], dtype=int8)

In [9]:
MultiDiscrete([5,2,2]).sample()

array([1, 0, 1], dtype=int64)

# 3. Building an Environment
- Build a agent to give us the best shower possible
- Randomly temperature
- 37 and 39 degrees

In [12]:
Box(low=0, high=100, shape=(1,)).sample()

array([80.5834], dtype=float32)

In [15]:
class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(3)
        self.observation_space = Box(low=0, high=100, shape=(1,))
        self.state = 38 + random.randint(-3, 3)
        self.shower_length = 60

    def step(self, action):
        self.state += action-1
        self.shower_length -= 1

        if self.state >= 37 and self.state <= 39:
            reward = 1
        else:
            reward = -1

        if self.shower_length <= 0:
            done = True
        else:
            done = False
        info = {}

        return self.state, reward, done, info

    def render(self):
        pass

    def reset(self):
        self.state = np.array([38 + random.randint(-1, 3)]).astype(float)
        self.shower_length = 60
        return self.state


In [16]:
env = ShowerEnv()

# 4. Test Environment

In [23]:
episodes = 5
for episode in range(1, episodes + 1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        score += reward
    print('Episode: {} Score: {}'.format(episode, score))
env.close()

Episode: 1 Score: -20
Episode: 2 Score: 6
Episode: 3 Score: -26
Episode: 4 Score: -60
Episode: 5 Score: -52


# 5. Train Model

In [24]:
log_path = os.path.join('Training', 'logs')
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [25]:
model.learn(total_timesteps=4000)


Logging to Training\logs\PPO_2
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -32.5    |
| time/              |          |
|    fps             | 180      |
|    iterations      | 1        |
|    time_elapsed    | 11       |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -32.1       |
| time/                   |             |
|    fps                  | 283         |
|    iterations           | 2           |
|    time_elapsed         | 14          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008969334 |
|    clip_fraction        | 0.0782      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | -0.000132   |

# 6. Save Model

In [26]:
ppo_path = os.path.join('Training', 'Saved Models', 'custom-environment-model')
model.save(ppo_path)


In [27]:
del model

In [31]:
model = PPO.load(ppo_path, env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [32]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

c:\Users\rober\Desktop\machine-learning\.venv\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


TypeError: render() got an unexpected keyword argument 'mode'